# ML Experiments using different bin sizes & widths based on outputs from supervised discretization
## Dataset: Pen Digits

by: Malina & Sam , 26.06.2022

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, recall_score, precision_score
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import six
import sys
sys.modules['sklearn.externals.six'] = six
import id3
from id3 import Id3Estimator
from id3 import export_graphviz
import wittgenstein as lw
import warnings
warnings.filterwarnings('ignore')

# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

## EDA
from collections import Counter

# Knn-VDM 3
from vdm3 import ValueDifferenceMetric
from sklearn.neighbors import KNeighborsClassifier

# Pre-processing
from sklearn.preprocessing import OrdinalEncoder
# Cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score # 1 metric
from sklearn.model_selection import cross_validate # more than 1 metric
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
#read in data from ChiMerge discretization
pen6 = pd.read_csv('chim_pen_6int.csv')
pen8 = pd.read_csv('chim_pen_8int.csv')
pen10 = pd.read_csv('chim_pen_10int.csv')
pen15 = pd.read_csv('chim_pen_15int.csv')

## Counting interval frequencies

In [3]:
num_list6 = pen6.columns.drop('class')
num_list8 = pen8.columns.drop('class')
num_list10 = pen10.columns.drop('class')
num_list15 = pen15.columns.drop('class')


In [4]:
print('Interval frequency for 6 Intervals')
for i in num_list6:
    print('Interval for ' + i)
    print(Counter(pen6[i]))

Interval frequency for 6 Intervals
Interval for A1
Counter({3: 2438, 0: 2364, 2: 2322, 4: 1413, 1: 1312, 5: 1143})
Interval for A2
Counter({5: 3115, 3: 2327, 4: 2054, 1: 1298, 2: 1215, 0: 983})
Interval for A3
Counter({3: 2917, 4: 2910, 0: 2115, 2: 1375, 1: 1138, 5: 537})
Interval for A4
Counter({5: 4387, 3: 2600, 1: 1655, 4: 899, 2: 799, 0: 652})
Interval for A5
Counter({5: 2567, 4: 2420, 1: 2408, 0: 1568, 3: 1224, 2: 805})
Interval for A6
Counter({4: 3709, 2: 2375, 3: 1813, 5: 1115, 1: 1037, 0: 943})
Interval for A7
Counter({3: 4001, 2: 2348, 0: 2227, 4: 1123, 5: 899, 1: 394})
Interval for A8
Counter({4: 2093, 0: 2034, 2: 1933, 1: 1774, 5: 1662, 3: 1496})
Interval for A9
Counter({1: 2823, 4: 1983, 0: 1765, 5: 1764, 2: 1611, 3: 1046})
Interval for A10
Counter({5: 2299, 0: 2239, 4: 1854, 1: 1797, 2: 1431, 3: 1372})
Interval for A11
Counter({5: 3645, 0: 2656, 4: 1843, 3: 1397, 1: 741, 2: 710})
Interval for A12
Counter({4: 3254, 2: 2392, 1: 1527, 5: 1306, 3: 1286, 0: 1227})
Interval for 

In [5]:
print('Frequency for iris with 8 Intervals')
for i in num_list8:
    print('Interval for ' + i)
    print(Counter(pen8[i]))

Frequency for iris with 8 Intervals
Interval for A1
Counter({0: 2364, 2: 2322, 3: 1672, 1: 1312, 7: 1143, 4: 766, 5: 750, 6: 663})
Interval for A2
Counter({7: 3115, 5: 2327, 6: 2054, 3: 1298, 4: 1215, 2: 720, 1: 234, 0: 29})
Interval for A3
Counter({0: 2115, 3: 1713, 5: 1712, 2: 1375, 4: 1204, 6: 1198, 1: 1138, 7: 537})
Interval for A4
Counter({7: 4387, 4: 1647, 1: 1105, 5: 953, 6: 899, 3: 799, 0: 652, 2: 550})
Interval for A5
Counter({7: 2567, 0: 1568, 6: 1452, 1: 1414, 4: 1224, 2: 994, 5: 968, 3: 805})
Interval for A6
Counter({5: 2737, 4: 1813, 3: 1788, 7: 1115, 1: 1037, 6: 972, 0: 943, 2: 587})
Interval for A7
Counter({5: 4001, 3: 1333, 1: 1201, 6: 1123, 0: 1026, 4: 1015, 7: 899, 2: 394})
Interval for A8
Counter({5: 2093, 3: 1933, 2: 1774, 0: 1556, 4: 1496, 6: 934, 7: 728, 1: 478})
Interval for A9
Counter({0: 1765, 7: 1764, 2: 1695, 3: 1611, 6: 1156, 1: 1128, 4: 1046, 5: 827})
Interval for A10
Counter({0: 2239, 1: 1797, 2: 1431, 3: 1372, 4: 1193, 7: 1161, 6: 1138, 5: 661})
Interval 

In [6]:
print('Frequency for iris with 10 Intervals')
for i in num_list10:
    print('Interval for ' + i)
    print(Counter(pen10[i]))

Frequency for iris with 10 Intervals
Interval for A1
Counter({0: 2364, 3: 2322, 9: 1143, 4: 1042, 6: 766, 7: 750, 2: 708, 8: 663, 5: 630, 1: 604})
Interval for A2
Counter({9: 3115, 5: 1568, 3: 1298, 4: 1215, 8: 1073, 7: 981, 6: 759, 2: 720, 1: 234, 0: 29})
Interval for A3
Counter({5: 1713, 7: 1712, 4: 1375, 6: 1204, 8: 1198, 1: 1162, 0: 953, 3: 612, 9: 537, 2: 526})
Interval for A4
Counter({9: 4387, 5: 1647, 2: 1105, 6: 953, 4: 799, 3: 550, 1: 486, 8: 451, 7: 448, 0: 166})
Interval for A5
Counter({0: 1568, 7: 1452, 8: 1412, 5: 1224, 9: 1155, 3: 994, 6: 968, 4: 805, 2: 712, 1: 702})
Interval for A6
Counter({7: 2737, 9: 1115, 1: 1037, 8: 972, 0: 943, 4: 929, 6: 917, 5: 896, 3: 859, 2: 587})
Interval for A7
Counter({5: 1853, 3: 1333, 6: 1317, 1: 1201, 8: 1123, 0: 1026, 4: 1015, 9: 899, 7: 831, 2: 394})
Interval for A8
Counter({4: 1933, 0: 1556, 6: 1524, 5: 1496, 3: 950, 8: 934, 2: 824, 9: 728, 7: 569, 1: 478})
Interval for A9
Counter({9: 1764, 3: 1695, 8: 1156, 2: 1128, 6: 1046, 0: 1028, 

In [7]:
print('Frequency for iris with 15 Intervals')
for i in num_list15:
    print('Interval for ' + i)
    print(Counter(pen15[i]))

Frequency for iris with 15 Intervals
Interval for A1
Counter({0: 2364, 6: 1228, 14: 1143, 7: 1042, 4: 977, 3: 708, 10: 685, 13: 663, 8: 630, 12: 597, 2: 460, 11: 153, 1: 144, 5: 117, 9: 81})
Interval for A2
Counter({14: 3115, 13: 1073, 12: 981, 10: 903, 11: 759, 9: 665, 7: 648, 8: 567, 6: 549, 4: 409, 3: 383, 5: 340, 2: 337, 1: 234, 0: 29})
Interval for A3
Counter({8: 1713, 10: 1712, 9: 1204, 0: 953, 7: 869, 11: 739, 3: 612, 5: 612, 4: 526, 6: 506, 12: 459, 2: 398, 14: 309, 13: 228, 1: 152})
Interval for A4
Counter({14: 4387, 4: 1105, 9: 971, 8: 676, 5: 550, 7: 545, 11: 478, 10: 475, 13: 451, 12: 448, 6: 254, 3: 245, 2: 241, 1: 90, 0: 76})
Interval for A5
Counter({0: 1568, 11: 1338, 14: 1155, 12: 1002, 9: 968, 3: 738, 2: 712, 1: 702, 8: 686, 7: 538, 5: 429, 13: 410, 6: 376, 4: 256, 10: 114})
Interval for A6
Counter({12: 1392, 11: 1345, 14: 1115, 13: 972, 10: 917, 9: 896, 6: 859, 3: 567, 0: 499, 8: 472, 2: 470, 7: 457, 1: 444, 4: 323, 5: 264})
Interval for A7
Counter({8: 1853, 0: 1026, 

In [8]:
pen8

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,class
0,3,7,2,4,4,1,2,0,0,2,4,5,7,7,4,7,8
1,0,5,2,7,3,5,3,4,1,1,3,0,5,1,7,1,2
2,0,2,2,2,6,5,7,7,4,7,3,4,0,3,2,0,1
3,0,7,0,5,1,4,1,4,5,3,7,4,6,3,5,0,4
4,0,3,4,4,7,7,5,6,3,6,3,4,1,3,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10987,2,7,1,3,0,1,4,3,6,4,5,5,6,3,7,0,4
10988,2,4,3,7,4,4,3,2,0,0,1,1,5,1,7,2,2
10989,4,7,2,4,0,1,1,0,3,1,7,4,6,7,4,6,0
10990,2,7,0,1,1,1,4,0,6,2,7,5,5,7,2,6,0


# 1. Categorical Naive Bayes

## 1.1 6 Intervals from ChiMerge

## 1.1 debug version

In [9]:
# dataX = pen6.drop('class', axis=1)
# dataY = pen6['class']

# train_ratio = 0.75
# validation_ratio = 0.15
# test_ratio = 0.10

# # train is now 75% of the entire data set
# # the _junk suffix means that we drop that variable completely
# # dataX, dataY: initial dataframe

# X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# # test is now 10% of the initial data set
# # validation is now 15% of the initial data set
# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

# #print(x_train, x_val, x_test)

In [10]:
# import time
# start = time.time() # For measuring time execution

# cnb = CategoricalNB()
# cnb.fit(X_train, y_train)

# y_pred_cnb = cnb.predict(X_test)
# y_prob_pred_cnb = cnb.predict_proba(X_test)

# end = time.time()
# print(end - start) # Total time execution for this sample


In [11]:
# # how did our model perform?
# count_misclassified = (y_test != y_pred_cnb).sum()
# print("CategoricalNB")
# print("=" * 25)
# print('Misclassified samples: {}'.format(count_misclassified))
# accuracy = accuracy_score(y_test, y_pred_cnb)
# print('Accuracy: {:.2f}'.format(accuracy))
# print("=" * 25)
# print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
# print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
# print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
# print("=" * 25)
# print("Classification report:")
# print(classification_report(y_test, y_pred_cnb))

In [12]:
# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

## 1.1 script version

In [13]:
# make test & train split
dataX = pen6.drop('class', axis=1)
dataY = pen6['class']

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# dataX, dataY: initial dataframe

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

#print(x_train, x_val, x_test)

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)


# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

#for validation
y_val_cnb = cnb.predict(X_val)
y_val_pred_cnb = cnb.predict_proba(X_val)
print("Classification report for validation:")
print(classification_report(y_val, y_val_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 145
Accuracy: 0.87
Recall score :  0.8681818181818182
Precision score :  0.8681818181818182
F1 score :  0.8681818181818183
Classification report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.91       102
           1       0.82      0.76      0.79       126
           2       0.89      0.92      0.90       123
           3       0.80      0.96      0.88        84
           4       0.99      0.97      0.98       115
           5       0.87      0.58      0.70       118
           6       0.98      0.96      0.97        99
           7       0.96      0.84      0.90       123
           8       0.82      0.89      0.86       100
           9       0.70      0.95      0.81       110

    accuracy                           0.87      1100
   macro avg       0.88      0.87      0.87      1100
weighted avg       0.88      0.87      0.87      1100

Classification report for validation:
              precis

In [14]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.132
Average bias: 0.129
Average variance: 0.015
Sklearn 0-1 loss: 0.132


In [15]:
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

Classification report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.91       102
           1       0.82      0.76      0.79       126
           2       0.89      0.92      0.90       123
           3       0.80      0.96      0.88        84
           4       0.99      0.97      0.98       115
           5       0.87      0.58      0.70       118
           6       0.98      0.96      0.97        99
           7       0.96      0.84      0.90       123
           8       0.82      0.89      0.86       100
           9       0.70      0.95      0.81       110

    accuracy                           0.87      1100
   macro avg       0.88      0.87      0.87      1100
weighted avg       0.88      0.87      0.87      1100



## 1.2 8 Intervals from ChiMerge

In [16]:
# make test & train split
dataX = pen8.drop('class', axis=1)
dataY = pen8['class']

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# dataX, dataY: initial dataframe

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

#validation select the model or tune the parameters
# 

In [17]:
import time
start = time.time() # For measuring time execution

# make test & train split
dataX = pen8.drop('class', axis=1)
dataY = pen8['class']

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# dataX, dataY: initial dataframe

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

#validation select the model or tune the parameters
# 

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))


#for validation
y_val_cnb = cnb.predict(X_val)
y_val_pred_cnb = cnb.predict_proba(X_val)
print("Classification report for validation:")
print(classification_report(y_val, y_val_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 142
Accuracy: 0.87
Recall score :  0.8709090909090909
Precision score :  0.8709090909090909
F1 score :  0.8709090909090909
Classification report:
              precision    recall  f1-score   support

           0       0.94      0.89      0.91       123
           1       0.83      0.62      0.71       117
           2       0.80      0.97      0.87       117
           3       0.83      0.94      0.88       107
           4       0.98      0.97      0.98       116
           5       0.87      0.62      0.73       111
           6       0.99      0.97      0.98       108
           7       0.96      0.89      0.92        98
           8       0.83      0.89      0.86        93
           9       0.75      0.96      0.84       110

    accuracy                           0.87      1100
   macro avg       0.88      0.87      0.87      1100
weighted avg       0.88      0.87      0.87      1100

Classification report for validation:
              precis

In [18]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.133
Average bias: 0.127
Average variance: 0.015
Sklearn 0-1 loss: 0.129


## 1.3 10 Intervals from ChiMerge

In [19]:
# make test & train split
dataX = pen10.drop('class', axis=1)
dataY = pen10['class']

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# dataX, dataY: initial dataframe

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

#validation select the model or tune the parameters
# 

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))


#for validation
y_val_cnb = cnb.predict(X_val)
y_val_pred_cnb = cnb.predict_proba(X_val)
print("Classification report for validation:")
print(classification_report(y_val, y_val_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 128
Accuracy: 0.88
Recall score :  0.8836363636363637
Precision score :  0.8836363636363637
F1 score :  0.8836363636363637
Classification report:
              precision    recall  f1-score   support

           0       0.94      0.89      0.92       116
           1       0.81      0.75      0.78       130
           2       0.81      0.93      0.87       114
           3       0.90      0.93      0.91       112
           4       0.99      0.98      0.99       107
           5       0.93      0.66      0.77       112
           6       1.00      0.98      0.99       105
           7       0.97      0.88      0.93       113
           8       0.85      0.91      0.88       110
           9       0.70      0.99      0.82        81

    accuracy                           0.88      1100
   macro avg       0.89      0.89      0.88      1100
weighted avg       0.89      0.88      0.88      1100

Classification report for validation:
              precis

In [20]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.119
Average bias: 0.115
Average variance: 0.019
Sklearn 0-1 loss: 0.116


## 1.4 15 Intervals from ChiMerge

In [21]:
# make test & train split
dataX = pen15.drop('class', axis=1)
dataY = pen15['class']

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# dataX, dataY: initial dataframe

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

#validation select the model or tune the parameters
# 

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))


#for validation
y_val_cnb = cnb.predict(X_val)
y_val_pred_cnb = cnb.predict_proba(X_val)
print("Classification report for validation:")
print(classification_report(y_val, y_val_cnb))

end = time.time()
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 117
Accuracy: 0.89
Recall score :  0.8936363636363637
Precision score :  0.8936363636363637
F1 score :  0.8936363636363637
Classification report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       123
           1       0.81      0.74      0.78       109
           2       0.83      0.96      0.89       118
           3       0.88      0.95      0.92       109
           4       0.98      0.96      0.97        96
           5       0.92      0.68      0.78       101
           6       0.98      0.99      0.99       104
           7       1.00      0.86      0.93       118
           8       0.87      0.89      0.88       114
           9       0.78      0.95      0.86       108

    accuracy                           0.89      1100
   macro avg       0.90      0.89      0.89      1100
weighted avg       0.90      0.89      0.89      1100

Classification report for validation:
              precis

In [22]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.104
Average bias: 0.105
Average variance: 0.017
Sklearn 0-1 loss: 0.106


# 2. Decision Tree ID3

## 2.1 6 Intervals from ChiMerge

### Debug version

In [23]:
# X = pena6.drop('label', axis=1)
# y = pen6['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)

In [24]:
# import time
# start = time.time() # For measuring time execution

# estimator = Id3Estimator()
# estimator = estimator.fit(X_train, y_train, check_input=True)
# tree = export_graphviz(estimator.tree_, 'tree.dot', y)

# y_pred_id3 = estimator.predict(X_test)
# #_prob_pred_id3 = estimator.predict_proba(X_test)

# end = time.time()
# print("Computation time:")
# print(end - start) # Total time execution for this sample

In [25]:
# accuracy = accuracy_score(y_test, y_pred_id3)
# print('Accuracy: {:.2f}'.format(accuracy))
# print("=" * 25)
# print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
# print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
# print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
# print("=" * 25)
# print("Classification report:")
# print(classification_report(y_test, y_pred_id3))

In [26]:
# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

### script version

In [27]:
# make test & train split
dataX = pen6.drop('class', axis=1)
dataY = pen6['class']

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# dataX, dataY: initial dataframe

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))

#for validation
y_val_id3 = estimator.predict(X_val)
print("Classification report for validation:")
print(classification_report(y_val, y_val_id3))


#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.95
Recall score :  0.9472727272727273
Precision score :  0.9472727272727273
F1 score :  0.9472727272727273
Classification report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       103
           1       0.88      0.88      0.88       105
           2       0.86      0.92      0.89       104
           3       0.98      0.96      0.97       112
           4       0.99      0.97      0.98       149
           5       0.94      0.97      0.96       102
           6       0.98      1.00      0.99       111
           7       0.96      0.93      0.95       121
           8       0.98      0.90      0.94        94
           9       0.93      0.93      0.93        99

    accuracy                           0.95      1100
   macro avg       0.95      0.95      0.95      1100
weighted avg       0.95      0.95      0.95      1100

Classification report for validation:
              precision    recall  f1-score   support

      

In [28]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.058
Average bias: 0.032
Average variance: 0.045
Sklearn 0-1 loss: 0.053


## 2.2 8 Intervals from ChiMerge

In [29]:
# make test & train split
dataX = pen8.drop('class', axis=1)
dataY = pen8['class']

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# dataX, dataY: initial dataframe

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))

#for validation
y_val_id3 = estimator.predict(X_val)
print("Classification report for validation:")
print(classification_report(y_val, y_val_id3))

#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.96
Recall score :  0.96
Precision score :  0.96
F1 score :  0.96
Classification report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       120
           1       0.92      0.91      0.91       116
           2       0.91      0.95      0.93       115
           3       1.00      0.98      0.99       110
           4       0.98      0.97      0.97       117
           5       0.97      1.00      0.99       106
           6       0.97      0.97      0.97        92
           7       0.98      0.95      0.96       125
           8       0.96      0.96      0.96       105
           9       0.96      0.91      0.93        94

    accuracy                           0.96      1100
   macro avg       0.96      0.96      0.96      1100
weighted avg       0.96      0.96      0.96      1100

Classification report for validation:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     

In [30]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.054
Average bias: 0.021
Average variance: 0.049
Sklearn 0-1 loss: 0.040


## 2.3 10 Intervals from ChiMerge

In [31]:
# make test & train split
dataX = pen10.drop('class', axis=1)
dataY = pen10['class']

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# dataX, dataY: initial dataframe

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))

#for validation
y_val_id3 = estimator.predict(X_val)
print("Classification report for validation:")
print(classification_report(y_val, y_val_id3))

#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.96
Recall score :  0.9590909090909091
Precision score :  0.9590909090909091
F1 score :  0.9590909090909091
Classification report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       130
           1       0.88      0.91      0.90       104
           2       0.93      0.91      0.92       113
           3       0.97      0.97      0.97       102
           4       0.99      0.97      0.98        99
           5       0.98      0.97      0.97        95
           6       1.00      0.99      1.00       126
           7       0.97      0.94      0.96       121
           8       0.96      0.97      0.96        96
           9       0.94      0.97      0.96       114

    accuracy                           0.96      1100
   macro avg       0.96      0.96      0.96      1100
weighted avg       0.96      0.96      0.96      1100

Classification report for validation:
              precision    recall  f1-score   support

      

In [32]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.049
Average bias: 0.020
Average variance: 0.042
Sklearn 0-1 loss: 0.041


## 2.4 15 Intervals from CHiMerge

In [33]:
# make test & train split
dataX = pen15.drop('class', axis=1)
dataY = pen15['class']

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
# dataX, dataY: initial dataframe

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))

#for validation
y_val_id3 = estimator.predict(X_val)
print("Classification report for validation:")
print(classification_report(y_val, y_val_id3))

#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.96
Recall score :  0.9645454545454546
Precision score :  0.9645454545454546
F1 score :  0.9645454545454546
Classification report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       119
           1       0.97      0.91      0.94       120
           2       0.92      0.97      0.94       102
           3       0.96      1.00      0.98        90
           4       0.98      0.98      0.98       124
           5       0.95      0.97      0.96       117
           6       0.97      0.98      0.98       106
           7       0.95      0.97      0.96       109
           8       0.98      0.95      0.97       118
           9       0.98      0.93      0.95        95

    accuracy                           0.96      1100
   macro avg       0.96      0.96      0.96      1100
weighted avg       0.97      0.96      0.96      1100

Classification report for validation:
              precision    recall  f1-score   support

      

In [34]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.041
Average bias: 0.012
Average variance: 0.036
Sklearn 0-1 loss: 0.035


# 3. KNN classification

In [35]:
#read in data from ChiMerge discretization
pen6 = pd.read_csv('chim_pen_6int.csv')
pen8 = pd.read_csv('chim_pen_8int.csv')
pen10 = pd.read_csv('chim_pen_10int.csv')
pen15 = pd.read_csv('chim_pen_15int.csv')

## 3.1 KNN with ChiMerge 6 Intervals

In [36]:
# Complete code for data preperation
# Read data
df_ewd1 = pen6.head(500)
disc = 'EWD'
k = 4

df_ewd1.info()
data = df_ewd1.values
data.shape

features = df_ewd1.drop('class', axis = 1).columns

# separate the data into X and y
X = data[:, : len(features)]
Y = data[:,-1]

print(X.shape, Y.shape)

# Split train test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30)

# Check representation of class
print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   A1      500 non-null    int64
 1   A2      500 non-null    int64
 2   A3      500 non-null    int64
 3   A4      500 non-null    int64
 4   A5      500 non-null    int64
 5   A6      500 non-null    int64
 6   A7      500 non-null    int64
 7   A8      500 non-null    int64
 8   A9      500 non-null    int64
 9   A10     500 non-null    int64
 10  A11     500 non-null    int64
 11  A12     500 non-null    int64
 12  A13     500 non-null    int64
 13  A14     500 non-null    int64
 14  A15     500 non-null    int64
 15  A16     500 non-null    int64
 16  class   500 non-null    int64
dtypes: int64(17)
memory usage: 66.5 KB
(500, 16) (500,)
Class representation - original:  Counter({0: 61, 5: 56, 6: 55, 3: 51, 2: 49, 4: 47, 1: 46, 7: 46, 8: 45, 9: 44})
Class representation - training data:  Counter({0: 49, 5: 4

In [40]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.86      1.00      0.92        12
           1       0.71      0.77      0.74        13
           2       0.82      0.90      0.86        10
           3       0.82      1.00      0.90         9
           4       1.00      1.00      1.00        12
           5       0.92      0.92      0.92        12
           6       1.00      1.00      1.00        17
           7       1.00      0.81      0.90        16
           8       1.00      0.73      0.84        11
           9       1.00      1.00      1.00        13

    accuracy                           0.91       125
   macro avg       0.91      0.91      0.91       125
weighted avg       0.92      0.91      0.91       125

Time for training model Knn-VDM: 90.7321572303772.


In [41]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, x_train, y_train, x_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

KeyboardInterrupt: 

## 3.2 KNN with ChiMerge 8 Intervals

In [ ]:
# Complete code for data preperation
# Read data
df_ewd1 = pen8.head(500)
disc = 'EWD'
k = 4

df_ewd1.info()
data = df_ewd1.values
data.shape

features = df_ewd1.drop('class', axis = 1).columns

# separate the data into X and y
X = data[:, : len(features)]
Y = data[:,-1]

print(X.shape, Y.shape)

# Split train test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30)

# Check representation of class
print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

In [ ]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

In [ ]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, x_train, y_train, x_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.3 KNN with ChiMerge 10 Intervals

In [ ]:
# Complete code for data preperation
# Read data
df_ewd1 = pen10.head(500)
disc = 'EWD'
k = 4

df_ewd1.info()
data = df_ewd1.values
data.shape

features = df_ewd1.drop('class', axis = 1).columns

# separate the data into X and y
X = data[:, : len(features)]
Y = data[:,-1]

print(X.shape, Y.shape)

# Split train test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30)

# Check representation of class
print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

In [ ]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

In [ ]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, x_train, y_train, x_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.4 KNN with ChiMerge 15 Intervals

In [ ]:
# Complete code for data preperation
# Read data
df_ewd1 = pen15.head(500)
disc = 'EWD'
k = 4

df_ewd1.info()
data = df_ewd1.values
data.shape

features = df_ewd1.drop('class', axis = 1).columns

# separate the data into X and y
X = data[:, : len(features)]
Y = data[:,-1]

print(X.shape, Y.shape)

# Split train test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30)

# Check representation of class
print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

In [ ]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

In [ ]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, x_train, y_train, x_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))